# In Verbis Virtus - ETL
## Create fact tables from data stored in PostgreSQL DBMS
Some initial imports and functions definitions

In [ ]:
import pandas as pd
import psycopg2 as ps
import math
import os

def get_data(conn, query):
    results = []
    try:
        cursor = conn.cursor()
        cursor.execute(query)
        for data in cursor.fetchall():
            results.append(data)
        cursor.close()
        print(f'Num of rows fetched: {len(results)}')
    except Exception as e:
        print(f"Exception: {e}")
    return results

def get_col_names(conn, table_str):

    col_names = []
    try:
        cursor = conn.cursor()
        cursor.execute(f"SELECT * FROM {table_str} LIMIT 0")
        for desc in cursor.description:
            col_names.append(desc[0])        
        cursor.close()
    except Exception as e:
        print(f"Exception: {e}")

    return col_names

def get_df_data(conn, query_str, table_str):
    col_names = get_col_names(conn, table_str)
    results = get_data(conn, query_str)
    df = pd.DataFrame(results, columns=col_names)
    return df

def distance_calculator(steps):
    tot_distance = 0.0
    for index, _ in steps.iterrows():
        if index < steps.shape[0] -2:
            x_diff = (steps.loc[index].posx - steps.loc[index+1].posx)
            y_diff = (steps.loc[index].posy - steps.loc[index+1].posy)
            distance = math.sqrt(x_diff**2+y_diff**2)
            tot_distance = tot_distance + distance
    return tot_distance


### Connect to DB
Pay attention to replace your data to create the right connection

In [ ]:
user = 'postgres'
password = 'testing'
host = 'localhost'
port = '54321'
db_name = 'IVV'

connection_string = f"user='{user}' password='{password}' host='{host}' port='{port}' dbname='{db_name}'"

try:
    conn = ps.connect(connection_string)
    print(f"Connection established: {conn}")
except Exception as e:
    print(f"Exception: {e}")



### Get tables names

In [ ]:
cursor = conn.cursor()
cursor.execute("""
    SELECT table_name
    FROM information_schema.tables
    WHERE table_schema = 'public'
""")

# Fetch all the table names
table_names = cursor.fetchall()
cursor.close()

# Print the table names
for table_name in table_names:
    print(table_name[0])

### List columns names from seleted table

In [ ]:
table_name = 'stg_ivv_analytics'
col_names = get_col_names(conn, table_name)
col_names


### Get unique players IDs
...and save as CSV

In [ ]:
table_name = 'stg_ivv_analytics'
query_str = f"SELECT DISTINCT player_id  FROM {table_name}"
players = get_data(conn, query_str)
players_df = pd.DataFrame(players, columns=['player'])

if not os.path.exists('fact/CSV'):
    os.makedirs('fact/CSV')

players_df.to_csv(f'./fact/CSV/players.csv')

#players

# Dataset
## Create dataset as CSV files for data visualization
The JSON file (see below) seems to be too large for Tableau desktop

In [ ]:
## limiting the number of players for testing
# players = players[0:50]

In [ ]:
table_name = 'stg_ivv_analytics'

i = 0
row_index = 0
sessions_df = pd.DataFrame()

for player in players:
    i = i + 1
    print(f'Working on {i}/{len(players)}')
    ## Create the DF to store all the steps to save as single CSV
    all_steps = pd.DataFrame()

    ## get the player_id
    player_id = player[0]   

    ## get user log from database
    query_str = f"SELECT * FROM {table_name} WHERE player_id ='{player_id}' ORDER BY session_id, log_id"
    rows = get_df_data(conn, query_str, table_name)

    ## set real_time and game_time to 0.0 for event_type 'BeginLog'
    rows.loc[rows.event_type =='BeginLog',['game_time','real_time']] = 0.0
    ## working on single session
    for session in rows.session_id.unique():
        #print(row_index)
        ## Create df and store some infos
        session_df = pd.DataFrame(columns=['player','session','session_logs','map_info','max_real_time','max_game_time','distance'])
        session_df.loc[row_index,'player'] = player_id
        session_df.loc[row_index,'session']  = session

        ## extract rows for current session
        session_rows = rows[rows.session_id == session]

        ## add data
        map_info = session_rows[session_rows.event_type == 'BeginLog'].map_info.item()
        session_df.loc[row_index,'session_logs'] = session_rows.shape[0]
        session_df.loc[row_index,'map_info'] = map_info
        session_df.loc[row_index,'max_real_time'] = session_rows.real_time.max()
        session_df.loc[row_index,'max_game_time'] =session_rows.game_time.max()
        
        ## extract steps where positions are defined
        steps = session_rows[~session_rows.position.isna()].reset_index()
        steps['map_info'] = map_info
        ## expand the x, y amd z position 
        steps[['posx','posy','posz']] = steps.position.str.split(',', expand=True)
        steps[['posx','posy','posz']] = steps[['posx','posy','posz']].astype(float)
        steps = steps.drop('position', axis=1)
        ## expand rotation infos
        steps[['pitch','yaw','roll']] = steps.rotation.str.split(',', expand=True)
        steps[['pitch','yaw','roll']] = steps[['pitch','yaw','roll']].astype(float)
        steps = steps.drop('rotation', axis=1)
        ## expand the x, y amd z velocity 
        steps[['velx','vely','velz']] = steps.velocity.str.split(',', expand=True)
        steps[['velx','vely','velz']] = steps[['velx','vely','velz']].astype(float)
        steps = steps.drop('velocity', axis=1)

        all_steps = pd.concat([all_steps, steps], ignore_index=True)

        ## calculate total distanstance for the player in the session
        session_df.loc[row_index,'distance'] = distance_calculator(steps)
        sessions_df = pd.concat([sessions_df, session_df], ignore_index=True)
        row_index = row_index + 1

    if not os.path.exists('fact/CSV/players'):
        os.makedirs('fact/CSV/players')
    
    all_steps.to_csv(f'./fact/CSV/players/player_{player_id}.csv')

sessions_df.to_csv(f'./fact/CSV/sessions.csv')


### Create a CSV with the all sessions details
This is needed for Tableau data visualization

In [ ]:
import warnings
warnings.filterwarnings('ignore')

folder = os.fsencode('./fact/CSV/players')
full_sessions_dataset = pd.DataFrame()

for file in os.listdir(folder):
    filename = os.fsdecode(file)
    print(filename)
    if filename.endswith(".csv") : 
        print(f'Working on {filename}')
        dataset = pd.read_csv(f'./fact/CSV/players/{filename}')
        full_sessions_dataset = pd.concat([full_sessions_dataset, dataset], ignore_index=True)
        print(f'Dataset shape {full_sessions_dataset.shape}')
        print('-*'*10)
    else:
        continue

full_sessions_dataset.to_csv(f'./fact/CSV/sessions_details.csv')

## Create the dataset as a Dictionary to store as a JSON

In [ ]:
table_name = 'stg_ivv_analytics'

data_list = []
i = 0
for player in players:
    i = i + 1
    print(f'Working on {i}/{len(players)}')
    ## get the player_id
    player_id = player[0]

    ## set some user infos into the user dictionary
    user_dict = {}
    user_dict['player'] = player_id
    user_dict['sessions'] = []

    ## get user log from database
    query_str = f"SELECT * FROM {table_name} WHERE player_id ='{player_id}' ORDER BY session_id, log_id"
    rows = get_df_data(conn, query_str, table_name)
    user_dict['total_logs'] = rows.shape[0]
    ## set real_time and game_time to 0.0 for event_type 'BeginLog'
    rows.loc[rows.event_type =='BeginLog',['game_time','real_time']] = 0.0
    
    ## working on single session
    for session in rows.session_id.unique():
        session_dict = {}
        ## extract rows for current session
        session_rows = rows[rows.session_id == session]

        ## add some infos to dict
        session_dict['id'] = session
        session_dict['session_logs'] = session_rows.shape[0]
        session_dict['map_info'] = session_rows[session_rows.event_type == 'BeginLog'].map_info.item()
        session_dict['max_real_time'] = session_rows.real_time.max()
        session_dict['max_game_time'] =session_rows.game_time.max()
        
        ## extract steps where positions are defined
        steps = session_rows[~session_rows.position.isna()].reset_index()
        ## expand the x, y amd z position 
        steps[['posx','posy','posz']] = steps.position.str.split(',', expand=True)
        steps[['posx','posy','posz']] = steps[['posx','posy','posz']].astype(float)
        steps = steps.drop('position', axis=1)
        ## expand rotation infos
        steps[['pitch','yaw','roll']] = steps.rotation.str.split(',', expand=True)
        steps[['pitch','yaw','roll']] = steps[['pitch','yaw','roll']].astype(float)
        steps = steps.drop('rotation', axis=1)
        ## expand the x, y amd z velocity 
        steps[['velx','vely','velz']] = steps.velocity.str.split(',', expand=True)
        steps[['velx','vely','velz']] = steps[['velx','vely','velz']].astype(float)
        steps = steps.drop('velocity', axis=1)

        ## extract other logs where positions are not defined
        other_log = session_rows[session_rows.position.isna()].reset_index()
        other_log = other_log.drop('health', axis=1)
        session_dict['other_logs']= []
        for log in other_log.to_dict('records'):
            session_dict['other_logs'].append( {k: v for k, v in log.items() if v is not None})

        ## set the steps and other logs under the session onject
        session_dict['steps'] = []
        for step in steps.to_dict('records'):
            session_dict['steps'].append({k: v for k, v in step.items() if v is not None})

        ## calculate total distanstance for the player in the session
        session_dict['distance'] = distance_calculator(steps)

        user_dict['sessions'].append(session_dict)

    data_list.append(user_dict)
    

### Save the dataset to a JSON file

In [ ]:
import json
import numpy as np

class NpEncoder(json.JSONEncoder):
    def default(self, obj):
        if isinstance(obj, np.integer):
            return int(obj)
        if isinstance(obj, np.floating):
            return float(obj)
        if isinstance(obj, np.ndarray):
            return obj.tolist()
        if isinstance(obj, pd.Timestamp):
            return obj.strftime('%Y-%m-%d %H:%M:%S')
        if isinstance(obj, type(pd.NaT)):
            return str('')
        return super(NpEncoder, self).default(obj)

json_str = json.dumps(data_list, cls=NpEncoder)

if not os.path.exists('fact'):
    os.makedirs('fact')

with open("./fact/ivv_fact.json", "w+") as f:
    json.dump(data_list, f, indent=4, sort_keys=False, cls=NpEncoder)